In [1]:

from dotenv import load_dotenv
load_dotenv()
import os
import sys
sys.path.append(
    os.environ.get('WORK_DIR')) #type: ignore
sys.path.append(
    os.environ.get('DBASE_DIR')) #type: ignore
from dbase.DataAPI.ThetaData import * #type: ignore
from dbase.database.SQLHelpers import * #type: ignore
import pandas as pd
from EventDriven.data import HistoricTradeDataHandler
from EventDriven.event import *
from queue import Queue
from trade.backtester_.backtester_ import PTDataset, PTBacktester
import pandas_ta as ta
from trade.assets.Stock import Stock
from trade.backtester_.utils.WalkForwardUtils import prev_monday 
from trade.backtester_.strats import BBandsTrend2
from trade.backtester_.strats import MAStrat
import yfinance as yf
from datetime import datetime
from EventDriven.backtest import OptionSignalBacktest
from copy import deepcopy
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2


Console Logging & File Logging Can be configured using STREAM_LOG_LEVEL and FILE_LOG_LEVEL in environment variables.
Propagate to root logger can be set using PROPAGATE_TO_ROOT_LOGGER in environment variables.
Example:
STREAM_LOG_LEVEL = 'DEBUG'
FILE_LOG_LEVEL = 'INFO'
PROPAGATE_TO_ROOT_LOGGER = 'False'

2025-03-11 16:34:21 trade.helpers.Logging INFO: Logging Root Directory: /Users/chiemelienwanisobi/cloned_repos/QuantTools/logs
Using Proxy URL: http://18.232.166.224:5500/thetadata


In [2]:
thetadata_start = '2021-01-01'

In [50]:
import json
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
key = 10
with open(f'../input/profitable_weights_{key}.json', 'r') as f:
    weights = json.load(f)
ttrades__ = pd.read_csv(f'../input/profitable_trades_{key}.csv').iloc[:, 1:]
ttrades__ = ttrades__[ttrades__.Ticker == 'NVDA']
ttrades__

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
4,151.0,518.0,753.0,18.979196,49.243999,4569.985385,1.59463,2023-01-25,2024-01-02,342 days,NVDA


In [51]:
trades_ = ttrades__.copy()
trades_

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration,Ticker
4,151.0,518.0,753.0,18.979196,49.243999,4569.985385,1.59463,2023-01-25,2024-01-02,342 days,NVDA


In [52]:
symbol_list = trades_.Ticker.unique()
untraded_symbols = [s for s in weights.keys() if s not in trades_.Ticker.unique()]
for s in untraded_symbols:
    weights.pop(s)
pd.Series(weights).sort_values(ascending=False)

NVDA    0.062106
dtype: float64

In [6]:

from EventDriven.riskmanager import RiskManager, close_cache, spot_cache, chain_cache, oi_cache, LOOKBACKS, order_cache
from pandas.tseries.offsets import BDay

rm = RiskManager(None, None, 1000000)
rm.OrderPicker.liquidity_threshold = 2
rm.OrderPicker.lookback = 10
rm.OrderPicker.data_availability_threshold = 0.15
date, tick = '2023-07-05', 'AVGO'
date, tick = '2024-08-13', 'TSM'
start = (pd.to_datetime(date) - BDay(30)).strftime('%Y-%m-%d')
right = 'C'
order_settings = {'type': 'spread',
 'specifics': [{'direction': 'long',
   'rel_strike': .50,
   'dte': 210,
   'moneyness_width': 0.5},
  #  {'direction': 'short',
  # 'rel_strike': .60,
  # 'dte': 270,
  # 'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}



order =  rm.OrderPicker.get_order(tick, date, right, 2, order_settings)
order

Saving to cache from db
Updating db rates data


{'result': 'SUCCESSFUL',
 'data': {'long': ['TSM20250321C280'],
  'trade_id': '&L:TSM20250321C280',
  'close': 1.515}}

In [53]:
max_cash = {}
cash = 20_000
for s, w in weights.items():
    if w * cash > 200:
        max_cash[s] = 2
    elif w * cash > 100:
        max_cash[s] = 1
    else:
        max_cash[s] = 0.5
max_cash

{'NVDA': 2}

In [130]:
#Backtest class 
## Find a way to not always reinitialize the backtest class, when want to redo
evb_backtest = OptionSignalBacktest(trades_, initial_capital=cash)

In [131]:
evb_backtest.portfolio.initial_capital

20000.0

In [132]:
w_map = {x: w  * 0.75 for x, w in weights.items()} ## 75% of the weights for each stock
evb_backtest.portfolio.weight_map = w_map
evb_backtest.portfolio.weight_map
evb_backtest.portfolio.risk_manager.OrderPicker.liquidity_threshold = 100
evb_backtest.portfolio.risk_manager.OrderPicker.lookback = 10
evb_backtest.portfolio.risk_manager.OrderPicker.data_availability_threshold = 0.5
evb_backtest.portfolio.order_settings = {'type': 'naked',
 'specifics': [{'direction': 'long',
   'rel_strike': .850,
   'dte': 300,
   'moneyness_width': 0.35},
   {'direction': 'short',
  'rel_strike': .60,
  'dte': 300,
  'moneyness_width': 0.35}
],
 'name': 'vertical_spread'}

# evb_backtest.portfolio.order_settings = {'type': 'naked',
#  'specifics': [{'direction': 'long',
#    'rel_strike': .750,
#    'dte': 210,
#    'moneyness_width': 0.5},

# ],
#  'name': 'vertical_spread'}


evb_backtest.portfolio.max_contract_price = max_cash
evb_backtest.executor.commission_rate = 0.65/100
evb_backtest.executor.commission_rate
evb_backtest.executor.max_slippage_pct = 0.075
evb_backtest.portfolio.roll_map = 180
evb_backtest.portfolio.allocated_cash_map
evb_backtest.portfolio.max_contract_price
evb_backtest.portfolio.roll_map

{'NVDA': 180}

In [133]:
signals = evb_backtest.bars.signal_df
# series = signals.set_index('Date')['MSFT']
# series[series != 0]
signals

,Date,NVDA
0,2023-01-25,1
1,2023-01-26,0
2,2023-01-27,0
3,2023-01-30,0
4,2023-01-31,0
...,...,...
240,2023-12-27,0
241,2023-12-28,0
242,2023-12-29,0
243,2024-01-01,0


In [134]:
signals_df = deepcopy(signals).set_index('Date')
signals_df[signals_df!=-1].sum().sum()

1.0

In [135]:
import cProfile
import pstats
import io

profiler = cProfile.Profile()
profiler.enable()
#run backtest

evb_backtest.run()
profiler.disable()
stream = io.StringIO()
stats = pstats.Stats(profiler, stream=stream).sort_stats('cumulative')

Processing event: MARKET
Processing event: SIGNAL
Processing event: ORDER
Processing event: FILL
2023-01-25 00:00:00
NVDA BUY
Event queue is empty, processed 4 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing event: MARKET
Event queue is empty, processed 1 event(s)
Processing even

In [14]:
for positio, whole in evb_backtest.portfolio._OptionSignalPortfolio__trades.items():
    if positio == '&L:MSFT20220916C370&S:MSFT20220916C385':
        continue
    # print(positio)
    try:
        (whole['pnl'])
    except:
        print(positio)

&L:NVDA20240920C950&S:NVDA20240920C1000
&L:COST20240920C840&S:COST20240920C850


In [136]:
evb_backtest.portfolio.plot_portfolio()

In [61]:
evb_backtest.portfolio.current_positions

{'NVDA': {}}

In [99]:
trades = evb_backtest.portfolio._trades.copy()#
trades[['Positions','Ticker','PnL', 'ReturnPct', 'EntryPrice', 'ExitPrice', 'EntryMarketValue', 'ExitMarketValue', 'EntryTime', 'ExitTime', 'EntryPrice', 'ExitPrice', 'Quantity', 'SignalID']]
# len(trades['SignalID'].unique())

,Positions,Ticker,PnL,ReturnPct,EntryPrice,ExitPrice,EntryMarketValue,ExitMarketValue,EntryTime,ExitTime,EntryPrice,ExitPrice,Quantity,SignalID
0,&L:NVDA20240119C380&S:NVDA20240119C450,NVDA,-94.383822,-0.115245,204.746078,181.150122,813.784311,729.800489,2023-01-25,2023-07-24,204.746078,181.150122,4,NVDA20230125LONG
1,&L:NVDA20240621C570&S:NVDA20240621C575,NVDA,-81.366026,-0.123412,164.826152,144.484646,654.104609,583.138583,2023-07-25,2023-12-26,164.826152,144.484646,4,NVDA20230125LONG
2,&L:NVDA20240920C780&S:NVDA20240920C800,NVDA,NaN,NaN,150.318978,NaN,149.018978,NaN,2023-12-26,NaT,150.318978,NaN,1,NVDA20230125LONG
3,&L:NVDA20240920C650&S:NVDA20240920C660,NVDA,-52.256999,-0.125768,207.750892,181.622393,412.901785,365.844786,2023-12-27,2024-01-02,207.750892,181.622393,2,NVDA20230125LONG


In [63]:
evb_backtest.portfolio.aggregate()

Start                                                    2023-01-25 00:00:00
End                                                      2024-01-02 00:00:00
Duration                                                   342 days 00:00:00
Exposure Time [%]                                                      100.0
Equity Final [$]                                                     19810.7
Equity Peak [$]                                                 36548.169533
Return [%]                                                          -0.83827
Buy & Hold Return [%]                                             146.178132
CAGR [%]                                                           -0.894392
Volatility Ann. [%]                                                54.862834
Sharpe Ratio                                                        0.319229
Sortino Ratio                                                       0.270096
Skew                                                               -6.331022

In [106]:
eq = evb_backtest.portfolio._equity.copy()
eq[eq.index >= '2023-07-20']

,NVDA,cash,commission,Total
datetime,,,,
2023-07-20,15872.606412,19068.409277,0.052,34941.015689
2023-07-21,15192.606412,19068.409277,0.052,34261.015689
2023-07-24,837.206901,19068.409277,0.104,19905.616178
2023-07-25,807.902292,19068.409277,0.156,19876.311569
2023-07-26,787.902292,19068.409277,0.156,19856.311569
...,...,...,...,...
2023-12-27,575.020112,19068.409277,0.299,19643.429389
2023-12-28,580.020112,19068.409277,0.299,19648.429389
2023-12-29,585.020112,19068.409277,0.299,19653.429389


In [109]:
#0	&L:NVDA20240119C380&S:NVDA20240119C450
price = (evb_backtest.portfolio.options_data['NVDA20240119C380'] - evb_backtest.portfolio.options_data['NVDA20240119C450']).copy()
price[price.index >= '2023-01-25']
price[price.index.isin( ['2023-01-25', '2023-07-24'])]

,Open,High,Low,Close,Volume,Bid_size,CloseBid,Ask_size,CloseAsk,Midpoint,Weighted_midpoint
Datetime,,,,,,,,,,,
2023-01-25,1.69,2.04,1.67,2.04,-5,-253,1.9,59,1.95,1.925,2.082202
2023-07-24,35.90,37.75,40.20,37.86,-183,127,36.7,94,39.80,38.250,38.037282


In [50]:

pd.set_option('display.max_rows', 10000)
evb_backtest.portfolio.get_all_positions()

long                short  \
datetime   symbol                                             
2022-01-04 AAPL     [AAPL20220916C210]   [AAPL20220916C220]   
           COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-05 AAPL     [AAPL20220916C210]   [AAPL20220916C220]   
           COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-06 COST     [COST20230120C680]   [COST20230120C700]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-07 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-10 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-11 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-12 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-13 COST     [COST20230120C560]   [COST20230120C570]   
           MU         [MU20220916C100]     [MU20220916C105]   
2022-01-14 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-17 MU         [MU20220916C100]     [MU20220916C105]   
2022-01-18 COST     [COST20230120C540]   [COST20230120C550]   
           MU         [MU20220916C100]     [MU20220916C105]   
           TSM       [TSM20220916C140]    [TSM20220916C145]   
2022-01-19 COST     [COST20230120C540]   [COST20230120C550]   
           HD         [HD20230120C420]     [HD20230120C430]   
           MU         [MU20220916C100]     [MU20220916C105]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
           TSM       [TSM20220916C140]    [TSM20220916C145]   
2022-01-20 HD         [HD20230120C420]     [HD20230120C430]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
2022-01-21 HD         [HD20230120C420]     [HD20230120C430]   
           BAC        [BAC20221021C42]     [BAC20221021C50]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
           MSFT     [MSFT20230120C385]   [MSFT20220916C365]   
2022-01-24 TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-25 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-26 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           TSM       [TSM20230120C150]    [TSM20220916C145]   
2022-01-27 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-01-28 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-01-31 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-01 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-02 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-03 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-04 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
           COST     [COST20230120C680]   [COST20230120C740]   
           HD         [HD20230120C450]     [HD20230120C460]   
           MSFT     [MSFT20230120C420]   [MSFT20230120C450]   
2022-02-07 AAPL     [AAPL20221118C200]   [AAPL20221118C300]   
  

In [373]:


""""
Why do these two return weird stuff after run?
    current_weighted_holdings
    current_positions

I can't reconcile the cost with the data (NVM, haha)

"""
# evb_backtest.portfolio.all_positions

'"\nWhy do these two return weird stuff after run?\n    current_weighted_holdings\n    current_positions\n\nI can\'t reconcile the cost with the data (NVM, haha)\n\n'

In [269]:
order_cache['2024-01-03']

{'BA': {'result': 'TOO_ILLIQUID', 'data': None},
 'BAC': {'result': 'SUCCESSFUL',
  'data': {'long': ['BAC20240719C32'],
   'trade_id': '&L:BAC20240719C32',
   'close': 3.2}},
 'INTC': {'result': 'SUCCESSFUL',
  'data': {'long': ['INTC20240719C39'],
   'trade_id': '&L:INTC20240719C39',
   'close': 12.25}},
 'HD': {'result': 'TOO_ILLIQUID', 'data': None},
 'AMD': {'result': 'SUCCESSFUL',
  'data': {'long': ['AMD20240719C135'],
   'trade_id': '&L:AMD20240719C135',
   'close': 16.625}},
 'QCOM': {'result': 'SUCCESSFUL',
  'data': {'long': ['QCOM20240719C140'],
   'trade_id': '&L:QCOM20240719C140',
   'close': 45.9}},
 'MU': {'result': 'SUCCESSFUL',
  'data': {'long': ['MU20240719C85'],
   'trade_id': '&L:MU20240719C85',
   'close': 29.375}}}

## Extend for  get_port_stats
- Buy & Hold
- Dates
- Trades
- _strategy in Aggregate
- The function

In [214]:
evb_backtest.trades.to_csv(f'/Users/chiemelienwanisobi/cloned_repos/QuantTools/EventDriven/output/profitable_trades_options_{key}.csv')